# Murat Gonul's Capstone Project Study

## description of problem

In Turkey, coffie shops are becoming more and more popular and it is becoming a race for two brands that are penetrating to Turkish market for the last couple of years. Nero and Starbucks are having branches in all over Istanbul. Aim of the study is finding out best approach to get Nero ahead in the competition by using Foursquare data.

## how data might with that aim

Based on foursquare data, density of shops can be compared between two brands to see if any local market is missed. Additionally, data can show competition in terms of rating. Based on these inputs, a recommendation to Nero can be made to take this company ahead in the competition.


In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

In [2]:
CLIENT_ID = 'TDLYPU2YXQQBECSDQ02CLIVWWVOAYG2MZB1THAR4N30FYJDI' # your Foursquare ID
CLIENT_SECRET = 'FLMVJU3X5EEKBQOHNWVIRVZUMBCWNOZGQY5VQ142UYYRTMWJ' # your Foursquare Secret
ACCESS_TOKEN = '2CRDYDMLUC0WO24ILPKQDIV3O3LYUTEOXU4STABDGUVQB02A' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TDLYPU2YXQQBECSDQ02CLIVWWVOAYG2MZB1THAR4N30FYJDI
CLIENT_SECRET:FLMVJU3X5EEKBQOHNWVIRVZUMBCWNOZGQY5VQ142UYYRTMWJ


In [3]:
address = 'taksim,istanbul'
search_query_nero = ['Nero']
search_query_starbucks = ['Starbucks']
radius = 1000
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.0380506 28.985549


In [4]:
url_nero = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query_nero, radius, LIMIT)
url_starbucks = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query_starbucks, radius, LIMIT)


In [5]:
results_nero = requests.get(url_nero).json()
results_starbucks = requests.get(url_starbucks).json()

In [6]:
venues_nero = results_nero['response']['venues']
venues_starbucks = results_starbucks['response']['venues']

In [7]:
# tranform venues into a dataframe
dataframe_nero = json_normalize(venues_nero)
dataframe_starbucks = json_normalize(venues_starbucks)

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_starbucks = ['name', 'categories'] + [col for col in dataframe_starbucks.columns if col.startswith('location.')] + ['id']
filtered_columns_nero = ['name', 'categories'] + [col for col in dataframe_nero.columns if col.startswith('location.')] + ['id']

dataframe_filtered_nero = dataframe_nero.loc[:, filtered_columns_nero]
dataframe_filtered_starbucks = dataframe_starbucks.loc[:, filtered_columns_starbucks]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_nero['categories'] = dataframe_filtered_nero.apply(get_category_type, axis=1)
dataframe_filtered_starbucks['categories'] = dataframe_filtered_starbucks.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_nero.columns = [column.split('.')[-1] for column in dataframe_filtered_nero.columns]
dataframe_filtered_starbucks.columns = [column.split('.')[-1] for column in dataframe_filtered_starbucks.columns]


In [9]:
dataframe_filtered_nero.shape
dataframe_filtered_starbucks.shape

(8, 16)

In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Kadıköy',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_nero.lat, dataframe_filtered_nero.lng, dataframe_filtered_nero.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
for lat, lng, label in zip(dataframe_filtered_starbucks.lat, dataframe_filtered_starbucks.lng, dataframe_filtered_starbucks.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Data
Blue dots are showing Nero and black dots are showing Starbucks branches in Taksim which is one of the most crowded veneu in Istanbul. Data can help with density of branches within main veneues.